In [ ]:
#This code to convert the xml files to excel files 
import numpy as np
from collections import defaultdict
from xml.dom import minidom
import glob
import os
import csv


index = ['at1.at','be1.be','ch1.ch','cz1.cz','de1.de','es1.es','fr1.fr','gr1.gr','hr1.hr','hu1.hu','ie1.ie','il1.il','it1.it','lu1.lu','nl1.nl','ny1.ny','pl1.pl','pt1.pt','se1.se','si1.si','sk1.sk','uk1.uk']
a = np.zeros((22, 22)) #create a matrix full of zeros with number of rows = number of sources and number of columns = number of targets
z = 0

MultiArray = [] #The Multi-dimentional array to store the demands between different cities

path = r'D:\Semester 1 Modules\Data Challenge\directed-geant-uhlig-15min-over-4months-ALL' #path for xml files folder

for filename in glob.glob("{0}\*".format(path)): #to iterate through all xml files

    if os.stat(filename).st_size/1024 < 3: #ignore empty files
        continue
    name = str(filename[-17:-4])
    doc = minidom.parse(filename)

    target = doc.getElementsByTagName('target') #get all <target> from xml
    source = doc.getElementsByTagName('source') #get all <source> from xml
    demandValue = doc.getElementsByTagName('demandValue') #get all <demandValue> from xml
    #time = doc.getElementsByTagName()

    v = []
    t = []
    s = []

    for elem in demandValue:
        v.append(elem.firstChild.data) #store all <demandValue> values in an array 'v'
    for elem in target:
        t.append(elem.firstChild.data) #store all <target> values in an array 't'
    for elem in source:
        s.append(elem.firstChild.data) #store all <source> values in an array 's'


    test = [[val] for val in s]
    for i in range(len(t)):
        test[i].append(t[i])
        test[i].append(v[i]) #create a nested list of [source, target, value] combination for each demandValue


    scol = np.array(index) #create matrix of unique sources
    scol = scol.reshape(-1, 1) #reshape source into a 2D column matrix

    trow = np.array(index) #create matrix of unique targets
    trow = trow.reshape(1, -1) #reshape target into a 2D row matrix
    trow = np.insert(trow, 0, 0) #add a zero as the first value of target

    Xnew = np.hstack((scol,a)) #add source as the first column to zero matrix
    Xnew = np.vstack([trow,Xnew]) #add target as the first row to zero matrix


    for i in range(len(test)):
        x= np.where(Xnew == test[i][0]) #get location of test source in the zero Matrix
        y = np.where(Xnew == test[i][1]) #get location of test target in zero Matrix
        j = x[0][1]
        k = y[0][1]
        Xnew[j][k]=test[i][2]
        X = np.delete(Xnew, 0, 1)

    MultiArray.append(X)
    
    #To save each xml file in a seperate excel file
    n = "./Results2/%s.csv" % str(filename[-17:-4])
    with open(n, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(MultiArray[z])
        
    print(z)

    z = z+1


In [ ]:
#This code gets the dates of all files and delete redundant values

import pandas as pd
import glob
import csv

#create an empty dataframe
df = pd.DataFrame()

#create an empty list for dates 
date = []

path = r'D:\Semester 1 Modules\Data Challenge\Results2'

for filename in glob.glob("{0}\*".format(path)):
    date.append(filename[-17:-9])

#only get uniqe values
res = []
[res.append(x) for x in date if x not in res]

print(res)



In [56]:
#This code gets the daily throughput adjaceny matrices 

import pandas as pd
import glob
import csv


#create an empty dataframe and date list
df = pd.DataFrame()
date = []


path = r'D:\Semester 1 Modules\Data Challenge\Results2'

for filename in glob.glob("{0}\*".format(path)):
    
    #check whether this date was calculated before 
    if filename[-17:-9] not in date:
        length = len(date)
        
        if length == 0:
            date.append(filename[-17:-9])
            df= pd.read_csv(filename)
            
        else:
            name = date[(length-1)]
            df.to_csv("D:/Semester 1 Modules/Data Challenge/Daily%s.csv" % name, index = True)
            date.append(filename[-17:-9])
            df = pd.DataFrame()
            df = pd.read_csv(filename) 
    else:
        df = df.add(pd.read_csv(filename), fill_value=0)
    
df.to_csv("D:/Semester 1 Modules/Data Challenge/Results2/Result/%s.csv" % filename[-17:-9], index = True)
